In [1]:
from models import *
from qhoptim.pyt import QHAdam
import pytorch_lightning as pl

In [2]:
import build
loader = build.dataloader(data='dataset_v6', sample_length=4096, part=0.001, batch_size=1, shuffle=True)
print(len(loader))

1


In [3]:
x, y = next(iter(loader))
print(x.shape, y.shape)

torch.Size([1, 256, 4096]) torch.Size([1])


In [4]:
print(x.min(), x.max())

tensor(0.) tensor(1.)


In [5]:
from models import *
from qhoptim.pyt import QHAdam
import pytorch_lightning as pl

import build
loader = build.dataloader(data='dataset_v6', sample_length=2**13, part=0.001, batch_size=1, shuffle=True)


In [6]:
premodel = MixtureNet(layers=3, blocks=3, res_channels=32, end_channels=32, classes=256, groups=16)

In [7]:
def NLL(y_hat, y):
    assert y_hat.shape == y.shape
    assert y_hat.dim() == 3 # (batch, channels, length)
    return -(y * torch.log(y_hat) + (1 - y) * torch.log(1 - y_hat)).sum(dim=1).mean()

class DaNet(pl.LightningModule):
    def __init__(self, model_loader, noise=0.0, lr=0.0):
        super().__init__()
        self.save_hyperparameters()
        self.model = eval(model_loader)
        self.lr = lr
        self.output_size = 2**12
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, aux = batch
        inp = F.pad(x, (1, -1)) + torch.randn_like(x) * self.hparams.noise
        x_hat = self.forward(inp)
        loss = NLL(x_hat[-self.output_size:], x[-self.output_size:])
        self.log("train_loss", loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = QHAdam(self.parameters(), lr=self.lr, nus=(0.7, 1.0))
        return {"optimizer": optimizer, "lr_scheduler": torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5**0.001)}

In [10]:
model = DaNet(module_description(premodel), noise=0.0, lr=1e-9)

In [11]:
trainer = pl.Trainer(log_every_n_steps=1)
trainer.fit(model, train_dataloaders=loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type       | Params
-------------------------------------
0 | model | MixtureNet | 11.8 K
-------------------------------------
11.8 K    Trainable params
0         Non-trainable params
11.8 K    Total params
0.047     Total estimated model params size (MB)


Epoch 10:   0%|          | 0/1 [00:00<?, ?it/s, loss=0.029, v_num=11]        

In [21]:
x = torch.randn((10, 256, 2**13))
y = model(x)
print(y.mean(), y.std())
t = y[0, :, -1].view(1, -1, 1)
# print(t)
d = F.one_hot(torch.tensor(36), 256).view(1, -1, 1)
print(NLL(t, d))

tensor(0.0039, grad_fn=<MeanBackward0>) tensor(0.0238, grad_fn=<StdBackward0>)
tensor(0.0290, grad_fn=<NegBackward0>)


In [ ]:
for p in model.parameters():
    print(p)